### 1. Reset the reference channel / threshold
1. Run the codes below to reset reference and threshold
2. Proceed to the next section

* **Restart the kernel when you move on to the next section**

In [ ]:
%matplotlib inline
%gui qt

import numpy as np
import matplotlib.pyplot as plt
from spiketag.base import probe
from spiketag.fpga import FPGA

# load probe
prb_filename = './prb_a2x32x2.prb'
prb = probe()
prb.load(prb_filename)

fpga = FPGA(probe=prb)

# check the current channel reference
ch_ref_fpga = fpga.ch_ref.to_numpy().astype(int)
print(f"Channel reference ordered by channel:")
for i in range(10):
    ch_str = ' '.join([f"{i:3d}" for i in ch_ref_fpga[i*16:(i+1)*16]])
    print(f"{i*16:3d}-{(i+1)*16:3d}: " + ch_str)
print(f"Channel reference ordered by probe location:")
for i in range(10):
    ch_str = ' '.join([f"{i:3d}" for i in ch_ref_fpga[prb.chs[i*16:(i+1)*16]]])
    print(f"{i*16:3d}-{(i+1)*16:3d}: " + ch_str)

# check the current threshold
thres = fpga.thres.to_numpy().copy()
plt.subplot(2, 1, 1)
plt.bar(np.arange(len(thres)), thres, width=1)
plt.hlines(-250, 0, len(thres), color='r', linestyles='dashed')
plt.ylabel('bit')
plt.xlabel('channel')
plt.xlim([-0.5, len(thres) - 0.5])
plt.ylim([-1000, 0])
plt.subplot(2, 1, 2)
plt.bar(np.arange(len(prb.chs)), thres[prb.chs] * 0.195, width=1)
plt.hlines(-50, 0, len(prb.chs), color='r', linestyles='dashed')
plt.vlines(np.arange(-0.5, 128, 32), 0, -200, color='r', linestyles='dashed')
plt.text(16, -100, 'prb 0\nshank 0', color='k', ha='center', va='center')
plt.text(16+32, -100, 'prb 0\nshank 1', color='k', ha='center', va='center')
plt.text(16+64, -100, 'prb 1\nshank 0', color='k', ha='center', va='center')
plt.text(16+96, -100, 'prb 1\nshank 1', color='k', ha='center', va='center')
plt.ylabel('uV')
plt.xlabel('sorted by probe position')
plt.xlim([-0.5, len(prb.chs) - 0.5])
plt.ylim([-200, 0])
plt.tight_layout()

print(thres[prb.chs])

In [ ]:
# probe channel information
probe0_shank0 = prb.chs[:32]
probe0_shank1 = prb.chs[32:64]
probe1_shank0 = prb.chs[64:96]
probe1_shank1 = prb.chs[96:128]

# reset reference channel
ch_ref_array = np.array(160*[160])

# reset threshold level
thres = np.ones(160) * -20000.0
thres[probe0_shank0] = -200.0
thres[probe0_shank1] = -200.0
thres[probe1_shank0] = -20000.0
thres[probe1_shank1] = -20000.0

fpga.set_channel_ref(ch_ref_array)
fpga.set_threshold(thres)

### 2. Select reference electrodes
1. Record for 1 minute (using open-ephys)
2. Run the following command to check the spike count per channel. **Choose silent channels** as the references per probe:
```bash
> spiketag report mua.bin spk.bin prb_a2x32x2.prb
```

3. Set the reference channel
4. Remove bin files (`rm *.bin`)
5. Proceed to the next section


In [ ]:
%matplotlib inline
%gui qt

import numpy as np
from spiketag.mvc import controller
from spiketag.base import probe

mua_filename = './mua.bin'
spk_filename = './spk.bin'
prb_filename = './prb_a2x32x2.prb'

# load probe
prb = probe()
prb.load(prb_filename)
probe0_shank0 = prb.chs[:32]
probe0_shank1 = prb.chs[32:64]
probe1_shank0 = prb.chs[64:96]
probe1_shank1 = prb.chs[96:128]

# load fpga
ctrl = controller(
                  fpga = True,
                  probe = prb,
                  mua_filename=mua_filename, 
                  spk_filename=spk_filename, 
                 )

# set reference channel
ch_ref_array = np.array(160*[160])

ref_probe0_shank0 = 38 # change this value
ref_probe0_shank1 = 38 # change this value
ref_probe1_shank0 = 160 # change this value
ref_probe1_shank1 = 160 # change this value

ch_ref_array[probe0_shank0] = ref_probe0_shank0
ch_ref_array[probe0_shank1] = ref_probe0_shank1
ch_ref_array[probe1_shank0] = ref_probe1_shank0
ch_ref_array[probe1_shank1] = ref_probe1_shank1

if ref_probe0_shank0 < 160:
    ch_ref_array[ref_probe0_shank0] = 160
if ref_probe0_shank1 < 160:
    ch_ref_array[ref_probe0_shank1] = 160
if ref_probe1_shank0 < 160:
    ch_ref_array[ref_probe1_shank0] = 160
if ref_probe1_shank0 < 160:
    ch_ref_array[ref_probe1_shank1] = 160

ctrl.fpga.set_channel_ref(ch_ref_array)
ch_ref_fpga = ctrl.fpga.ch_ref.to_numpy().astype(int)
print(f"Channel reference ordered by channel:")
for i in range(10):
    ch_str = ' '.join([f"{i:3d}" for i in ch_ref_fpga[i*16:(i+1)*16]])
    print(f"{i*16:3d}-{(i+1)*16:3d}: " + ch_str)
print(f"Channel reference ordered by probe location:")
for i in range(10):
    ch_str = ' '.join([f"{i:3d}" for i in ch_ref_fpga[prb.chs[i*16:(i+1)*16]]])
    print(f"{i*16:3d}-{(i+1)*16:3d}: " + ch_str)

### 3. Set threshold
1. Record for 1 minute
2. Set threshold
3. Remove bin files
4. Record about 10 minutes for clustering

In [ ]:
%matplotlib inline
%gui qt

import numpy as np
import matplotlib.pyplot as plt
from spiketag.mvc import controller
from spiketag.base import probe

mua_filename = './mua.bin'
spk_filename = './spk.bin'
prb_filename = './prb_a2x32x2.prb'

# load probe
prb = probe()
prb.load(prb_filename)
probe0_shank0 = prb.chs[:32]
probe0_shank1 = prb.chs[32:64]
probe1_shank0 = prb.chs[64:96]
probe1_shank1 = prb.chs[96:128]

# load fpga
ctrl = controller(
                  fpga = True,
                  probe = prb,
                  mua_filename=mua_filename, 
                  spk_filename=spk_filename, 
                  scale=False
                 )

# set threshold
ctrl.set_threshold(beta=4.5)

# if the reference is 160, we will turn it off.
ch_ref = ctrl.fpga.ch_ref.to_numpy().astype(int)
for i_ch, ch in enumerate(ch_ref):
    if ch == 160:
        ctrl.fpga.thres[i_ch] = -20000.

In [ ]:
# plot threshold
import matplotlib.pyplot as plt

thres = ctrl.fpga.thres.to_numpy().copy()

plt.subplot(2, 1, 1)
plt.bar(np.arange(len(thres)), thres, width=1)
plt.hlines(-250, 0, len(thres), color='r', linestyles='dashed')
plt.ylabel('bit')
plt.xlabel('channel')
plt.xlim([-0.5, len(thres) - 0.5])
plt.ylim([-1000, 0])

plt.subplot(2, 1, 2)
plt.bar(np.arange(len(prb.chs)), thres[prb.chs] * 0.195, width=1)
plt.hlines(-50, 0, len(prb.chs), color='r', linestyles='dashed')
plt.vlines(np.arange(-0.5, 128, 32), 0, -200, color='r', linestyles='dashed')
plt.text(16, -100, 'prb 0\nshank 0', color='k', ha='center', va='center')
plt.text(16+32, -100, 'prb 0\nshank 1', color='k', ha='center', va='center')
plt.text(16+64, -100, 'prb 1\nshank 0', color='k', ha='center', va='center')
plt.text(16+96, -100, 'prb 1\nshank 1', color='k', ha='center', va='center')
plt.ylabel('uV')
plt.xlabel('sorted by probe position')
plt.xlim([-0.5, len(prb.chs) - 0.5])
plt.ylim([-200, 0])

plt.tight_layout()

thres[prb.chs]

In [ ]:
# manually set threshold if needed (these values are in bits not in uV)
thres = np.ones(160, dtype=float) * -20000.0
thres[probe0_shank0] = -40.0
thres[probe0_shank1] = -500.0

# if the reference is 160, we will turn it off.
ch_ref = ctrl.fpga.ch_ref.to_numpy().astype(int)
for i_ch, ch in enumerate(ch_ref):
    if ch == 160:
        thres[i_ch] = -20000.0

ctrl.fpga.set_threshold(thres)
print(thres)

### 4. Spike sort
#### 1) Run background cluster
```bash
> ipcluster start --n=8
```
- You don't need this if you are doing only manual curation

#### 2) Open sorter
```python
>>> ctrl.show()
```

- Group selection plot
	- k, j: up, down
	- o: open
	- #g: go to cluster #
	- #bg: do gaussian clustering for the selected group (ex. 4bg, 5bg ...)
	- #bh: do hdbscan clustering for the selected group
	- z: undo
	- u: reset
	- d: done
- Waveform plot
	- c: select single spike
	- a: select all
	- left click: merge
	- right click: switch
	- esc: cancel selection

In [ ]:
%matplotlib inline
%gui qt

import os

from spiketag.mvc.Control import controller
from spiketag.base import probe

mua_filename = './mua.bin'
spk_filename = './spk.bin'
prb_filename = './prb_a2x32x2.prb'

prb = probe()
prb.load(prb_filename)
ctrl = controller(
                  fpga = True,
                  probe = prb,
                  mua_filename=mua_filename, 
                  spk_filename=spk_filename,
                  binary_radix=13, 
                  scale=False
                  # time_segs=[[0,320]] # if you want to load only part of the data
                 )

In [ ]:
# Open the sorter GUI
ctrl.show()

In [ ]:
# Save the model
ctrl.save('model')

In [ ]:
# Copy model to FPGA
ctrl.compile()

In [ ]:
print(ctrl.fpga.label)

### 5. Check clustered units to select target unit
- Check the pattern of each unit, and select the one that is stable, and well separated from the noise level

In [ ]:
from nctrl.unit import Unit

unit = Unit()
unit.load()

In [ ]:
unit.plot()

In [ ]:
unit.simulate(unit_id=1)

### 6. BMI
#### 1) If you are using FPGA's TTL output, go to section #4 and run
```python
>>> ctrl.fpga.target_unit = 1
```

Then open spiketag bmi
```bash
> spiketag bmi
```

#### 2) If you are using software close-loop system, run the section below.

In [ ]:
from nctrl import NCtrl

bmi = NCtrl()

In [ ]:
bmi.show()

### If the device is busy...
```bash
> fuser /dev/xillybus_template_32 -k
> fuser /dev/xillybus_fet_clf_32 -k
```

In [ ]:
# if the device is busy
!fuser /dev/xillybus_template_32 -k
!fuser /dev/xillybus_fet_clf_32 -k